In [1]:
import pandas as pd
import numpy as np
import sklearn
import math as mt
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

from operator import itemgetter
from sklearn import metrics
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

In [2]:
def DCGp(i, p, rel):
    tot = 0
    for i in range(p):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [3]:
def IDCGp(i, rel):
    tot = 0
    for i in range(abs(rel)):
        tot += (2**rel[i]-1)/mt.log2((i+1)+1)
    return tot

In [4]:
def nDCGp(i, p, rel):
    return DCGp(i, p, rel)/IDCGp(i, rel)

In [5]:
DF = pd.read_csv("citeulike-a/raw-data.csv",encoding='ISO-8859-1')#'ISO-8859-1')

In [6]:
DF.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,..."
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...


In [7]:
DF.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...


In [8]:
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 5 columns):
doc.id          16980 non-null int64
title           16980 non-null object
citeulike.id    16980 non-null float64
raw.title       16980 non-null object
raw.abstract    16980 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 663.4+ KB


In [9]:
citations = pd.read_csv("citeulike-a/citations.dat", header = None, delimiter='\n')

In [10]:
citations.columns = ["citations"]

In [11]:
citations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16980 entries, 0 to 16979
Data columns (total 1 columns):
citations    16980 non-null object
dtypes: object(1)
memory usage: 132.7+ KB


In [12]:
citations.head()

,citations
0,3 2 485 3284
1,16 42 43 60 113 116 161 252 1543 1782 1947 414...
2,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
3,0
4,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...


In [13]:
citations2=citations[1:len(citations)-1].copy()

In [14]:
#citations2.index.rename('doc.id',inplace=True)

In [15]:
citations2.reset_index(inplace=True)

In [16]:
citations2.tail()

,index,citations
16973,16974,12 1631 10468 10587 11949 12665 13198 14527 14...
16974,16975,16 420 6357 6371 9466 12007 12096 12660 13307 ...
16975,16976,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,0


In [17]:
DF2=pd.concat([DF,citations2],axis=1)
DF2

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...
5,6,random graphs with arbitrary degree distributi...,48.0,Random graphs with arbitrary degree distributi...,Recent work on the structure of social network...,6.0,14 49 1533 3646 7689 8302 10608 11649 12757 12...
6,7,artificial gene networks for objective compari...,49.0,Artificial gene networks for objective compari...,Motivation: Large-scale gene expression profil...,7.0,1 169
7,8,the segment polarity network is a robust devel...,50.0,The segment polarity network is a robust devel...,"All insects possess homologous segments, but s...",8.0,0
8,9,the evolutionary origin of complex features,52.0,The evolutionary origin of complex features.,A long-standing challenge to evolutionary theo...,9.0,9 3738 3757 4263 4298 6043 7014 9311 10171 15763
9,10,early language acquisition cracking the speech...,60.0,Early language acquisition: cracking the speec...,"Infants learn language with remarkable speed, ...",10.0,10 2 15 477 1200 1217 3937 5058 5561 8649 8958


In [18]:
type(DF2['citations'][0])

str

In [19]:
N = 10

In [20]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,1.0,16 42 43 60 113 116 161 252 1543 1782 1947 414...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,2.0,85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 46...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",3.0,0
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,4.0,23 2 28 488 918 1200 1351 1861 1937 2289 3079 ...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,5.0,37 2 15 22 23 28 254 469 477 481 483 484 485 4...


In [21]:
DF2.tail()

,doc.id,title,citeulike.id,raw.title,raw.abstract,index,citations
16975,16976,life is physics evolution as a collective phen...,8273254.0,Life is physics: evolution as a collective phe...,Evolution is the fundamental physical process ...,16976.0,8 7991 15184 15944 16272 16304 16451 16619 16722
16976,16977,limitations of nextgeneration genome sequence ...,8290615.0,Limitations of next-generation genome sequence...,High-throughput sequencing technologies promis...,16977.0,14 418 10406 10558 11796 14709 15093 15248 152...
16977,16978,accurate inference of transcription factor bin...,8305197.0,Accurate inference of transcription factor bin...,Accurate functional annotation of regulatory e...,16978.0,0
16978,16979,software that goes with the flow in systems bi...,8331304.0,Software that goes with the flow in systems bi...,ABSTRACT: A recent article in BMC Bioinformati...,NaN,NaN
16979,16980,a bacterium that can grow by using arsenic ins...,8349297.0,A Bacterium That Can Grow by Using Arsenic Ins...,Life is mostly composed of the elements carbon...,NaN,NaN


In [22]:
DF2.dropna(inplace=True,axis=0)

In [23]:
DF2['citations']=DF2['citations'].apply(lambda x: x.split(','))

In [24]:
del DF
del citations
del citations2

In [25]:
IdFreq={}

In [26]:
def docIdFreq (c_list):
    for element in c_list:
        if element in IdFreq:
            IdFreq[element] = IdFreq[element]+1
        else:
            IdFreq[element] = 1

In [27]:
DF2['citations'].apply(lambda x: docIdFreq(x));

In [28]:
most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)

In [29]:
del most_popular[0]

In [30]:
most_popular[0:N]

[('1 2216', 8),
 ('1 13117', 7),
 ('1 11229', 5),
 ('1 243', 4),
 ('1 3803', 4),
 ('1 1878', 4),
 ('1 5943', 4),
 ('1 965', 4),
 ('1 8717', 4),
 ('1 10913', 3)]

In [31]:
DF2.drop('index',axis=1,inplace=True)

In [32]:
DF2.head()

,doc.id,title,citeulike.id,raw.title,raw.abstract,citations
0,1,the metabolic world of escherichia coli is not...,42.0,The metabolic world of Escherichia coli is not...,To elucidate the organizational and evolutiona...,[16 42 43 60 113 116 161 252 1543 1782 1947 41...
1,2,reverse engineering of biological complexity,43.0,Reverse Engineering of Biological Complexity,Advanced technologies and biology have extreme...,[85 0 4 5 10 11 15 23 27 28 48 52 79 106 368 4...
2,3,exploring complex networks,44.0,Exploring complex networks,"The study of networks pervades all of science,...",[0]
3,4,comparative assessment of largescale data sets...,46.0,Comparative assessment of large-scale data set...,Comprehensive protein protein interaction maps...,[23 2 28 488 918 1200 1351 1861 1937 2289 3079...
4,5,navigation in a small world,47.0,Navigation in a small world,The small-world phenomenon â the principle t...,[37 2 15 22 23 28 254 469 477 481 483 484 485 ...


In [33]:
DF2[DF2['doc.id']==1]['raw.title'].values[0]

'The metabolic world of Escherichia coli is not small'

# Most popular article 

'The metabolic world of Escherichia coli is not small'

In [79]:
UR = pd.read_csv("citeulike-a/users.dat",delimiter='\n',names=['liked_articles'],header=None)

In [80]:
UR.head()

,liked_articles
0,70 495 1631 2317 2526 2846 2931 3171 3297 3332...
1,38 493 942 1519 1843 1844 1896 2819 3391 5031 ...
2,20 517 791 800 1329 1767 1984 2126 3009 4458 4...
3,12 706 709 721 755 756 776 892 895 2588 2991 4...
4,12 761 2678 4034 4035 5439 5494 5758 7798 9046...


In [81]:
UR['liked_articles']=UR['liked_articles'].apply(lambda x: x.split(' '))

In [82]:
UR.head()

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."


In [83]:
UR.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5551 entries, 0 to 5550
Data columns (total 1 columns):
liked_articles    5551 non-null object
dtypes: object(1)
memory usage: 43.4+ KB


In [39]:
kf = KFold(n_splits=5)
recommends = []

for train_index, validate_index in kf.split(UR):        
    trainDF = pd.DataFrame(UR.loc[train_index])
    validateDF = pd.DataFrame(UR.loc[validate_index])
    
    IdFreq={}
    trainDF['liked_articles'].apply(lambda x: docIdFreq(x));
    most_popular = sorted(IdFreq.items(), key=lambda x:x[1],reverse=True)
    del most_popular[0]
    most_popular = most_popular[0:N]
    mp_docId = [ element[0] for element in most_popular]
    recommends.append(DF2[DF2['doc.id'].isin(mp_docId)]['raw.title'].values)
    
    


In [40]:
recommends

[array(['Organization, development and function of complex brain networks',
        'Motifs in brain networks.',
        'Diffusion on Complex Networks : A way to probe their large scale topological structures',
        'Topological Generalizations of network motifs',
        "Collective dynamics of 'small-world' networks.",
        'Network motifs: simple building blocks of complex networks.',
        'Authoritative sources in a hyperlinked environment',
        'Finding and evaluating community structure in networks',
        'Subgraph centrality in complex networks.',
        'Finding structure with randomness: Stochastic algorithms for constructing approximate matrix decompositions'], dtype=object),
 array(['Organization, development and function of complex brain networks',
        'Motifs in brain networks.',
        'Diffusion on Complex Networks : A way to probe their large scale topological structures',
        'Topological Generalizations of network motifs',
        "Collectiv

# Random

In [41]:
d_id=np.random.randint(low=0, high=len(DF2)-1,size=N)

In [42]:
DF2[DF2['doc.id'].isin(d_id)]['raw.title'].values

array(['The polyadic pi-calculus: a tutorial',
       'Dynamical and correlation properties of the Internet',
       'The origin of bursts and heavy tails in human dynamics',
       'The Constitution of Society: Outline of the Theory of Structuration',
       'Quantum Computation',
       'The Optical Properties of Metal Nanoparticles:â\x80\x89 The Influence of Size, Shape, and Dielectric Environment',
       'Adaptive control of virtualized resources in utility computing environments',
       'Orthologous transcription factors in bacteria have different functions and regulate different genes.',
       'Quantum State Transfer and Entanglement Distribution among Distant Nodes in a Quantum Network',
       'Personalized tag recommendation using graph-based ranking on multi-type interrelated objects'], dtype=object)

# Content-Based

In [43]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english')

In [44]:
tfidf_matrix = tfidf_vectorizer.fit_transform(DF2['raw.abstract'])
nbrs_tfidf        = NearestNeighbors(n_neighbors=N+1,metric='cosine').fit(tfidf_matrix)

In [45]:
def get_closest_neighbors_tfidf(title):
    row                = DF2[DF2['raw.title']==title].index[0]
    distances, indices = nbrs_tfidf.kneighbors(tfidf_matrix.getrow(row))
    titles_similar      = pd.Series(indices.flatten()).map(DF2.reset_index()['raw.title'])
    result             = pd.DataFrame({'distance':distances.flatten(), 'raw.title':titles_similar})
    return result

In [46]:
get_closest_neighbors_tfidf('What is the goal of sensory coding')['raw.title'].values[1:N+1]

array([ 'Emergence of simple-cell receptive field properties by learning a sparse code for natural images',
       'Relations between the statistics of natural images and the response properties of cortical cells',
       'Sparse coding with an overcomplete basis set: A strategy employed by V',
       'Sparse coding and decorrelation in primary visual cortex during natural vision',
       'Natural image statistics and efficient coding',
       "The `Independent Components' of Natural Scenes are Edge Filters",
       'Cortical Sensitivity to Visual Features in Natural Scenes',
       'Statistical learning of new visual feature combinations by infants.',
       'Sparse Coding of Sensory Inputs',
       'Adaptive filtering enhances information transmission in visual cortex.'], dtype=object)

# Collaborative FiItering

In [47]:
docId = DF2['doc.id']

In [48]:
docId[len(docId)-1]

16978

In [49]:
del CountVectorizer
del Counter
del IdFreq
del LogisticRegression
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

In [50]:
DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

In [51]:
M = np.zeros(shape=(len(UR),len(docId)))

for i, row in UR.iterrows():
    for j in range(len(row['liked_articles'])-1):
        M[i,int(row['liked_articles'][j])-1]=1

In [52]:
BARRIER();

NameError: name 'BARRIER' is not defined

In [84]:
UR = UR.iloc[0:3999]

In [92]:
UR

,liked_articles
0,"[70, 495, 1631, 2317, 2526, 2846, 2931, 3171, ..."
1,"[38, 493, 942, 1519, 1843, 1844, 1896, 2819, 3..."
2,"[20, 517, 791, 800, 1329, 1767, 1984, 2126, 30..."
3,"[12, 706, 709, 721, 755, 756, 776, 892, 895, 2..."
4,"[12, 761, 2678, 4034, 4035, 5439, 5494, 5758, ..."
5,"[31, 648, 649, 661, 670, 689, 690, 691, 809, 8..."
6,"[12, 3292, 3579, 3917, 6247, 6723, 7492, 7495,..."
7,"[10, 1545, 2526, 5384, 8898, 9422, 11105, 1132..."
8,"[34, 861, 2215, 2461, 3082, 3445, 3446, 3515, ..."
9,"[26, 746, 830, 2000, 2095, 2439, 2476, 3090, 3..."


In [85]:
from scipy.sparse.linalg import svds

In [96]:
kf = KFold(n_splits=5)
recommendations = []
Users_index=[]
RecList=[]

for train_index, validate_index in kf.split(UR):
    
    UR_train = pd.DataFrame(UR.loc[train_index])
    UR_test = pd.DataFrame(UR.loc[validate_index])
    
    print(len(UR_train))
    
    M = np.zeros(shape=(len(UR_train),len(docId)))
    for i, row in UR_train.iterrows():
        print(i)
        for j in range(len(row['liked_articles'])-1):
            p=i-1
            print(i,p)
            M[p,int(row['liked_articles'][j])-1]=1

    U, sigma, Vt = svds(M, k = 50)

    sigma = np.diag(sigma)

    print(U.shape,Vt.shape)

    all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))

    preds_df = pd.DataFrame(all_user_predicted_ratings)

    for user_row_number in UR_test.index:

        sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

        recommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].
        merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
                       left_on = 'doc.id',
                       right_on = 'index'))

        recommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)

        recommendations.drop('index',axis=1,inplace=True)

        recommendations.sort_values('Predictions', ascending = False,inplace=True)

        Users_index.append(user_row_number)
        RecList.append(recommendations)

3199
800
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
800 799
801
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
801 800
802
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
802 801
803
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
803 802
804
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
804 803
805
805 804
805 804
805 804
805

845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
845 844
846
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
846 845
847
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
847 846
848
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
848 847
849


879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
879 878
880
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
880 879
881
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
881 880
882
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
882 881
883
883 882
883 882
883 882
883 882
883 882
883 882
883 882
883 882
883 882
883 882
883 882
884
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
884 883
885
885 884
885 884


937 936
937 936
937 936
937 936
937 936
937 936
938
938 937
938 937
938 937
938 937
938 937
938 937
938 937
938 937
938 937
938 937
939
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
939 938
940
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
940 939
941
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
941 940
942
942 941
942 941
942 941
942 941
942 941
942 941
942 941
942 941
942 941
942 

974 973
975
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
975 974
976
976 975
976 975
976 975
976 975
976 975
976 975
976 975
976 975
976 975
976 975
976 975
976 975
977
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 976
977 

...

3026 3025
3026 3025
3026 3025
3026 3025
3026 3025
3026 3025
3026 3025
3027
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3027 3026
3028
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3028 3027
3029
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3029 3028
3030
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029
3030 3029


3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073
3074 3073


3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3130 3129
3131
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3131 3130
3132
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131
3132 3131


3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3162 3161
3163
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 3162
3163 

IndexError: index 3199 is out of bounds for axis 0 with size 3199

In [ ]:
BARRIER();

In [ ]:
#del UR
del docId

In [ ]:
user_ratings_mean = np.sum(M, axis = 1)

In [ ]:
from scipy.sparse.linalg import svds

U, sigma, Vt = svds(M, k = 800)

In [ ]:
sigma = np.diag(sigma)

In [ ]:
print(U.shape,Vt.shape)

In [ ]:
all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))# + 
                              #user_ratings_mean.reshape(-1, 1))/user_ratings_mean.reshape(-1, 1)

In [ ]:
preds_df = pd.DataFrame(all_user_predicted_ratings)

# nDCG

## Recommending for user #:

In [ ]:
user_row_number=599

In [ ]:
sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

In [ ]:
preds_df.head()

In [ ]:
recommendations = (DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].
merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'doc.id',
               right_on = 'index'))

In [ ]:
recommendations.rename(columns = {user_row_number: 'Predictions'},inplace=True)

In [ ]:
recommendations.drop('index',axis=1,inplace=True)

In [ ]:
recommendations.sort_values('Predictions', ascending = False,inplace=True)

In [ ]:
BARRIER();

In [ ]:
# k = 200
recommendations.head(N)

In [ ]:
C=recommendations['doc.id'].head(10).tolist()

In [ ]:
A=UR['liked_articles'].iloc[user_row_number]

In [ ]:
sorted(C)

In [ ]:
DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].index

In [ ]:
A = list(map(int, A))

In [ ]:
sorted(A)

In [ ]:
set(C) & set(DF2[~DF2['doc.id'].isin(UR.iloc[user_row_number].values[0])].index)

In [ ]:
# k = 500
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 85
recommendations.head(N)

In [ ]:
# k = 80
recommendations.head(N)

In [ ]:
# k = 75
recommendations.head(N)

In [ ]:
# k=50
recommendations.head(N)

In [ ]:
# k=100
recommendations.head(N)

In [ ]:
# k=400
recommendations.head(N)

In [ ]:
# k=300
recommendations.head(N)

In [ ]:
# k=200
recommendations.head(N)

In [ ]:
recommendations.head(N)

In [ ]:
recommendations.head(N)

In [ ]:
recommendations.head(N)

docId = DF2['doc.id']

docId[len(docId)-1]

del CountVectorizer
del Counter
del IdFreq
del LogisticRegression
del TfidfVectorizer
del tfidf_matrix
del nbrs_tfidf
del tfidf_vectorizer

DF2.drop(['citeulike.id','title','citations'],axis=1,inplace=True)

M = np.zeros(shape=(len(UR),len(docId)))

for i, row in UR.iterrows():
    for j in range(len(row['liked_articles'])-1):
        M[i,int(row['liked_articles'][j])]=1

BARRIER();

#del UR
del docId

user_ratings_mean = np.sum(M, axis = 1)

from scipy.sparse.linalg import svds

U, sigma, Vt = svds(M, k = 500)

sigma = np.diag(sigma)

print(U.shape,Vt.shape)

all_user_predicted_ratings = (np.dot(np.dot(U, sigma), Vt))# + 
                              #user_ratings_mean.reshape(-1, 1))/user_ratings_mean.reshape(-1, 1)

preds_df = pd.DataFrame(all_user_predicted_ratings)

## Recommending for user #:

user_row_number=598

sorted_user_predictions = preds_df.iloc[user_row_number].sort_values(ascending=False)

preds_df.head()